# Hybrid Control Car

This project will demonstrated a hybrid machine learning and optimal control theory approach to controlling a simulated racecar. A neural network will be trained using reinforcement learning to pathfind, selecting the position of and the desired velocity of the car at the next path node based on sensor readings and the current state of the car. An inner control loop will use model predictive control with LQR feedback to drive the car to the next node.

The underlying physics simulation is written in the Rust programming language using [Rapier](https://rapier.rs/) as the physics engine with Python bindings generated by [PyO3](https://pyo3.rs). Its source code is avaiable at [src/lib.rs](src/lib.rs).

We will first start by importing the necessary libraries and establishing a few parameters.

In [1]:
from hybrid_control_car import CarSimulation # The simulation developed for this project
import numpy as np
import pysindy

hz = 60 # The number of simulation timesteps per second

# Modelling the System

While an accurate model of the car itself could be derived mathematically, for the purposes of this project we will not attempt to do so, treating the car as a standing for a far more complex system. To that end, we will use Sparse Indentification of Nonlinear Dynamics (SINDy) to create a model of the system computationally using the pySINDy library. This algorithm was chosen for several reasons:

1. It may be difficult to linearize about an equilibrium point while still maintaining a high level of accuracy.
2. pySINDy allows for finding the nonlinear dynamics using multiple trajectories of test data.
3. []

A model with at least 75% accuracy for the first few seconds is desired. Later on, feedback control will be used to compensate for any inaccuracies in the model, so a near-perfect model is not necessary.

In [5]:
# Set up the simulation
sim = CarSimulation()
sim.create_floor()

def generate_training_data(sim: CarSimulation, nsamples, nstates, inputs):
    data = np.zeros((nsamples, nstates))

    for n in range(nsamples):
        engine_force = inputs[n, 0]
        steering_angle = inputs[n, 1]
        state, _, _, _ = sim.step(engine_force, steering_angle)
        state = np.array(state)
        data[n, :] = state

    return data

nsamples = hz*10 # 10 seconds worth of sample data
nstates = 4
inputs = np.ones((nsamples, nstates))
data = generate_training_data(sim, hz*10, nstates, inputs)

print(data.shape)


(600, 4)


Foo

In [ ]:
model = pysindy.SINDy(feature_names=["x", "z", "vx", "vz", "u1", ])
model.fit()